In [536]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
         'Élément', 'Code Produit', 'Produit', 'Code Année', 'Année', 'Unité',
         'Valeur', 'Symbole', 'Description du Symbole']


fn_population = 'fao_2013/FAOSTAT_2013_population.csv'
fn_animal = 'fao_2013/FAOSTAT_2013_animal.csv'
fn_vegetal = 'fao_2013/FAOSTAT_2013_vegetal.csv'

animaux = pd.read_csv(fn_animal, names=names, header=0)
vegetaux = pd.read_csv(fn_vegetal, names=names, header=0)
df_pop = pd.read_csv(fn_population)
df_pop = df_pop[df_pop['Country'] != 'China']
pop_tot = 10**3*df_pop['Value'].sum()
print(pop_tot)

# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Code Produit',
    'Origine',
    'Élément',
    'Valeur',
    'Année'
]]

# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Code Produit','Année','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, df_pop[['Country', 'Value']], left_on='Pays', right_on='Country')

# Renommer les colonnes
aliments.rename(columns={
    'Aliments pour animaux' : 'alim_ani',
    'Autres utilisations (non alimentaire)' : 'autres_utilisations',
    'Semences' : 'semences',
    'Pertes' : 'pertes',
    'Nourriture' : 'nourriture',
    'Traitement' : 'transfo', # ?
    'Disponibilité intérieure' : 'dispo_int',
    'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',
    'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr',
    'Value' : 'population'
}, inplace=True)

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
aliments = aliments[aliments['Pays'] != 'Chine']
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')

# Création d'un fichier csv de la table aliments

#Calcul de la disponibilité en kcal par an pour tous les habitants du pays 
aliments['Disponibilté alimentaire en kcal par an'] = aliments['dispo_alim_kcal_p_j'] * (aliments['population']) * 365

#Disponibilité des protéines en kg par an pour tous les habitants du pays
aliments['Disponibilté proteines en kg par an'] = (aliments['dispo_prot']/1000 )* (aliments['population'])*365
aliments.sample(3)


6997326000
Résultat :  23 colonnes,  4764 lignes


Index(['Code Pays', 'Pays', 'Produit', 'Code Produit', 'Année', 'Origine',
       'alim_ani', 'Autres Utilisations', 'dispo_alim_kcal_p_j',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'dispo_mat_gr', 'dispo_prot', 'dispo_int', 'Exportations - Quantité',
       'Importations - Quantité', 'nourriture', 'pertes', 'Production',
       'semences', 'transfo', 'Variation de stock', 'Country', 'population',
       'Disponibilté alimentaire en kcal par an',
       'Disponibilté proteines en kg par an'],
      dtype='object')

In [537]:
#Calcul du ratio en énergie et le poids par an pour chaque pays
aliments['Ratio/Energie/Poids'] = (aliments['dispo_alim_kcal_p_j'] * 365) / aliments['Disponibilité alimentaire en quantité (kg/personne/an)']
aliments.replace(np.nan, 0, inplace=True)
ratio = aliments[aliments['Produit'] == 'Oeufs'].drop_duplicates('Produit')
ratio[['Produit','Ratio/Energie/Poids']]


,Produit,Ratio/Energie/Poids
37,Oeufs,1403.846154


In [538]:
# question 5 : 5 aliments les plus calorique
df_max_cal = aliments[['Produit', 'Ratio/Energie/Poids']]
df_max_cal = df_max_cal.sort_values('Ratio/Energie/Poids', ascending=False)
df_max_cal.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_cal = df_max_cal.dropna(subset=['Ratio/Energie/Poids'])
df_max_cal = df_max_cal.drop_duplicates('Produit')
df_max_cal.head(5)

,Produit,Ratio/Energie/Poids
1703,Huil Plantes Oleif Autr,18250.0
1540,Huile d'Olive,18250.0
725,Huile de Tournesol,18250.0
3507,Huiles de Foie de Poisso,18250.0
4182,Huile d'Arachide,18250.0


In [539]:
# question 5 : 5 aliments les plus riches en protéines
# 4 g de protéines = 4 kcal
df_max_prot = aliments[['Produit', 'Ratio/Energie/Poids', 'Disponibilté alimentaire en kcal par an', 'Disponibilté proteines en kg par an']]
df_max_prot['densite_prot'] = 4000*(df_max_prot['Disponibilté proteines en kg par an'] / df_max_prot['Disponibilté alimentaire en kcal par an']) * df_max_prot['Ratio/Energie/Poids']
df_max_prot = df_max_prot.sort_values('densite_prot', ascending=False)
df_max_prot.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_prot = df_max_prot.dropna(subset='densite_prot')
df_max_prot = df_max_prot.drop_duplicates('Produit')
df_max_prot.head(5)

/tmp/ipykernel_4611/2583843667.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_prot['densite_prot'] = 4000*(df_max_prot['Disponibilté proteines en kg par an'] / df_max_prot['Disponibilté alimentaire en kcal par an']) * df_max_prot['Ratio/Energie/Poids']


,Produit,Ratio/Energie/Poids,Disponibilté alimentaire en kcal par an,Disponibilté proteines en kg par an,densite_prot
577,Soja,3842.105263,77758140.0,7775.81400,1536.842105
619,Graines Colza/Moutarde,5840.000000,205462880.0,12841.43000,1460.000000
3153,"Plantes Oleiferes, Autre",3250.000000,864268535.0,84432.38765,1270.000000
4677,Arachides Decortiquees,6636.363636,1547600.0,67.70750,1161.363636
3273,"Viande, Autre",1079.881657,7051800.0,1621.91400,993.491124


In [540]:
# question 6 : disponibilité mondiale en millier de tonne (masse)
# poids moyen d'un être humain : 62 kg (masse )
# besoin calorique = metabolisme de base * 1.37 ~ 2000 kcal (énergie)
# masse moyenne théorique de population mondiale = 62 * pop_tot = 6997326000 * 62 = 433834212000 kg
df_dispo_mond = aliments[['Produit', 'Origine', 'dispo_int','Ratio/Energie/Poids']]
df_dispo_mond = df_dispo_mond.loc[df_dispo_mond['Origine'] == 'Végétale']
df_dispo_mond['dispo_int_kcal'] = 10**6 * df_dispo_mond['dispo_int'] * df_dispo_mond['Ratio/Energie/Poids']
df_dispo_mond.replace([np.inf, -np.inf], np.nan, inplace=True)
df_dispo_mond = df_dispo_mond.dropna(subset=['dispo_int_kcal'])
# Question 7
cal_h_an = (2000 * 365)
prot_h_an = (62 * 365)/1000
dispo_mond = df_dispo_mond['dispo_int_kcal'].sum()
dispo_prot = aliments['Disponibilté proteines en kg par an'].sum()

dispo_int_nourr = dispo_mond / cal_h_an
dispo_int_prot = dispo_prot / prot_h_an

print(f"{dispo_mond/10**15} Pcal")
print(f"{dispo_prot/10**6} MT")
print(f"{dispo_int_nourr / 10**9} milliards d'être huamin")
print(f"{100*dispo_int_nourr / pop_tot} %")
print(f"{100*dispo_int_prot / pop_tot} %")
dispo_mond

1.7638848157977345 Pcal
30.6340300164 MT
2.4162805695859375 milliards d'être huamin
34.531484878451245 %
0.019345834661836214 %


1763884815797734.5

In [541]:
# Question 8 : Potentiel alimentaire des végétaux
# Résultat : impact important légérement inférieur à la 
df_dispo_alim = aliments[['Produit', 'pertes', 'alim_ani', 'Disponibilté alimentaire en kcal par an', 'Disponibilité alimentaire en quantité (kg/personne/an)']]
df_dispo_alim['dispo_alim_kg'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)'] * aliments['population']
dispo_alim = df_dispo_alim['dispo_alim_kg'].sum()
alim_ani = 10**6*aliments['alim_ani'].sum()
pertes = 10*6*aliments['pertes'].sum()
sum_dispos = dispo_alim + alim_ani + pertes
print(sum_dispos / 10**16) # résultat incohérent

1.770248518114e-05


/tmp/ipykernel_4611/2044316073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dispo_alim['dispo_alim_kg'] = aliments['Disponibilité alimentaire en quantité (kg/personne/an)'] * aliments['population']


In [542]:
# Question 9 

df_dispo_mond = aliments[['Produit','Origine','Disponibilté proteines en kg par an','Disponibilté alimentaire en kcal par an','Ratio/Energie/Poids', 'pertes', 'alim_ani']]
dispo_alim_mond = df_dispo_mond['Disponibilté alimentaire en kcal par an'].sum()
dispo_prot_mond = df_dispo_mond['Disponibilté proteines en kg par an'].sum()

print(f"dispo alimentaire mondiale totale : {dispo_alim_mond / 10**16} Pkcal par an")
print(f"dispo protéique alimentaire mondiale : {dispo_prot_mond / 10**6} kg par an")
print(f"Pourcentage de la population concernée : {(dispo_alim_mond / (cal_h_an*pop_tot))*100} %")
print(f"Pourcentage de la population concernée : {(dispo_prot_mond / (prot_h_an*pop_tot))*100} %")

dispo alimentaire mondiale totale : 0.000114709983793 Pkcal par an
dispo protéique alimentaire mondiale : 30.6340300164 kg par an
Pourcentage de la population concernée : 0.022456716194157594 %
Pourcentage de la population concernée : 0.019345834661836214 %


In [543]:
# Question 10 : sous nutrition (done)
df_malnutrition = pd.read_csv('fao_2013/FAOSTAT_2013_sous_alimentation.csv', names=names, header=0)
df_malnutrition = df_malnutrition[df_malnutrition.Pays != 'Chine']
sum_malnutrition = df_malnutrition['Valeur'].sum()
print(f"{100 * (sum_malnutrition * 10**6 / pop_tot)} % de la population mondiale souffre de sous nutrition")

10.571181048303309 % de la population mondiale souffre de sous nutrition


In [544]:
# Question 11 : Etude sur les céréales (done)
df_cereal = pd.read_csv('fao_2013/FAOSTAT_2013_cereal.csv', names=names, header=0)
cereals = df_cereal[['Élément', 'Code Produit', 'Produit', 'Valeur']]
cereals = cereals.pivot_table('Valeur', index=['Code Produit','Produit'], columns='Élément')
cereals = cereals[['Nourriture','Aliments pour animaux']]
cereals['Propor_anim'] = 100 * (cereals['Aliments pour animaux'] / (cereals['Nourriture'] + cereals['Aliments pour animaux']))
percent_anim = 100 * cereals['Aliments pour animaux'].sum() / (cereals['Nourriture'].sum() + cereals['Aliments pour animaux'].sum())

print(f"{round(percent_anim)} % de la disponibilité alimentaire céréalière est consacrée à la nourriture des animaux")
cereals

50 % de la disponibilité alimentaire céréalière est consacrée à la nourriture des animaux


,Élément,Nourriture,Aliments pour animaux,Propor_anim
Code Produit,Produit,,,
2511,Blé,3126.914286,1563.620000,33.335648
2513,Orge,43.931250,785.491525,94.703395
2514,Maïs,788.315789,4267.792683,84.408646
2515,Seigle,43.261538,90.305263,67.610560
2516,Avoine,26.063291,136.186992,83.936366
2517,Millet,367.292308,32.848000,8.209120
2518,Sorgho,412.825397,200.044776,32.640645
2520,"Céréales, Autres",33.502924,141.697842,80.877410
2805,Riz (Eq Blanchi),2782.920000,557.451220,16.688301


In [545]:
# Question 12 (incomplet)
df_malnutrition = df_malnutrition[df_malnutrition['Valeur'].notna()]
df_malnutrition = df_malnutrition[['Code Pays', 'Pays', 'Valeur']]
df_malnutrition = pd.merge(aliments, df_malnutrition, on=['Code Pays', 'Pays'])
df_malnutrition.rename(columns={'Valeur_y':'malnutrition'}, inplace=True)
df_malnutrition = df_malnutrition[['Pays','Produit','Exportations - Quantité']].sort_values('Exportations - Quantité', ascending=False)
df_malnutrition = df_malnutrition.drop_duplicates('Produit')
df_malnutrition.head(15)


,Pays,Produit,Exportations - Quantité
3225,Viet Nam,Manioc,8973.0
3250,Viet Nam,Riz (Eq Blanchi),6951.0
2386,Paraguay,Soja,5117.0
2408,Philippines,Bananes,3268.0
2357,Paraguay,Maïs,2880.0
601,Costa Rica,Ananas,2301.0
1135,Guatemala,Sucre Eq Brut,1969.0
355,Myanmar,Haricots,1370.0
3194,Viet Nam,Café,1302.0
3243,Viet Nam,Poissons Eau Douce,1148.0
